Libraries

In [2]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import re
import time
from csv import writer, reader
from decimal import *
import pandas as pd
from tqdm import tqdm

Global variables

In [3]:
SEASONS = list(range(1981, 2023))
months = ['november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']

In [4]:
DATA_DIR = "testing_data_collection"
STANDINGS_DIR = os.path.join(DATA_DIR, "games")
curr_dir = '/Users/chrisjeff/Desktop/Origin'
#SCORES_DIR = os.path.join(DATA_DIR, "scores")

Function that grabs specified html content from a url

In [5]:
#get html function
async def get_html(url, selector, sleep=5, retries=10, timeout=0):
  html = None
  for i in range(1, retries+1):
    time.sleep(sleep * i)

    try:
      async with async_playwright() as p:
        browser = await p.firefox.launch()
        page = await browser.new_page()
        await page.goto(url, timeout=30000) #if can't load page in 30s, timeout error gets thrown
        print(url)
        html = await page.inner_html(selector,timeout=60000) #grab all html with identifier named {selector} within 60s
        print('got html')
    except PlaywrightTimeout:
      print(f"Timeout on {url}")
      if i == retries: #timesout for 5th time
        print('No games this month')
      continue
    else:
      break
  return html

Grabs html page containing all player names, positions, etc.

In [10]:
for season in SEASONS:
  url = f'https://basketball.realgm.com/nba/players/{season}'
  html = await get_html(url, 'table', timeout=60000)
  print(f'collected data from season {season}')

  soup = BeautifulSoup(html, 'html.parser')

  name_html = soup.find_all('a')
  pos_html = soup.find_all('td', attrs={'data-th': 'Pos'})

  names = []
  positions = []

  for row in name_html:
    names.append(row.text)
  print(len(names))
  print(names)

  for row in pos_html:
    positions.append(row.text)
  print(len(positions))
  print(positions)
'''
  with open(f'{curr_dir}/indv_player_data/season_{season}_players.csv', 'w') as f_object:
    writer_object = writer(f_object)
    assert len(names) == len(positions)
    for i in range(len(names)):
      writer_object.writerow(names[i])
      writer_object.writerow(positions[i])
    f_object.close()
'''

Timeout on https://basketball.realgm.com/nba/players/1981
Timeout on https://basketball.realgm.com/nba/players/1981
Timeout on https://basketball.realgm.com/nba/players/1981
Timeout on https://basketball.realgm.com/nba/players/1981
Timeout on https://basketball.realgm.com/nba/players/1981
https://basketball.realgm.com/nba/players/1981
got html
collected data from season 1981
1547
['Kareem Abdul-Jabbar', 'LAL', 'UCLA', '1969 Rnd 1 Pick 1', 'United States', 'Tom Abernethy', 'GOS', 'IND', 'Indiana', '1976 Rnd 3 Pick 9', 'United States', 'Alvan Adams', 'PHX', 'Oklahoma', '1975 Rnd 1 Pick 4', 'United States', 'Darrell Allums', 'DAL', 'UCLA', '1980 Rnd 5 Pick 11', 'United States', 'Tiny Archibald', 'BOS', 'UTEP', '1970 Rnd 2 Pick 2', 'United States', 'Dennis Awtrey', 'SEA', 'Santa Clara', '1970 Rnd 3 Pick 12', 'United States', 'James Bailey', 'SEA', 'Rutgers', '1979 Rnd 1 Pick 6', 'United States', 'Greg Ballard', 'WAS', 'Oregon', '1977 Rnd 1 Pick 4', 'United States', 'Mike Bantom', 'IND', '1

CancelledError: 

Gets links for stats to every game between a range of months, between a range of years

In [9]:

for season in SEASONS:
    for month in months:
        file_of_links = f'games_from_{month}-{season}.csv'
        if file_of_links in os.listdir(f'{curr_dir}/Collecting_Data/Links_to_games'):
            continue
        url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games-{month}.html'
        html = await get_html(url, '#schedule') #goes to url and returns all html
        if not html: #if html=none then a timeout occured, multiple times
            continue

        soup = BeautifulSoup(html, 'html.parser')
        row = soup.find_all('a')
        links = [_['href'] for _ in row] #grabs all links on a page
        all_links = [f"https://basketball-reference.com{game}" for game in links]

        indiv_game_boxscores = [] 
        for url in all_links:
            if 'boxscores' in url and url[-5::] == '.html':
                indiv_game_boxscores.append(url)
        print(indiv_game_boxscores) #list of all links to indiv games
        with open(f'{curr_dir}/Collecting_Data/Links_to_games/{file_of_links}', 'w') as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(indiv_game_boxscores)
            f_object.close()


https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
Timeout on https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
Timeout on https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
Timeout on https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
Timeout on https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
Timeout on https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
Timeout on https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
https://www.basketball-reference.com/leagues/NBA_1981_games-june.html
Timeout on https://www.b

Grabs the tables from a given web page

In [6]:
async def get_game_html_data(url):
    html = await get_html(url, '#content') #grabs div with id named content 
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all('table', class_='sortable stats_table') #tables is a list of html <table>
    return tables
    

Collect HTML for indv games

In [7]:
#Should be over 32881 game_html files
get_data = f'{curr_dir}/Collecting_Data' #directory of where collected data is put
html_game_dir = f'{get_data}/game_html' #directory of where html of games are put
for file_number, file_name in enumerate(os.listdir(f'{get_data}/Links_to_games')):
  print(file_name)
  all_links = []
  with open(f'{get_data}/Links_to_games/{file_name}', 'r') as file_object:
    csv_file = reader(file_object)

    all_links = list(csv_file)[0] #all_links is a list of links to games for a given month

  for single_link in all_links:
    new_file = single_link[-17::] #sets name of new_file to year/month/day/team of a game
    if new_file in os.listdir(f'{html_game_dir}'):
      continue
    print(f'visiting {single_link}')
    with open(f'{html_game_dir}/{new_file}', 'w') as f_object:
      html = await get_game_html_data(single_link)
      #if not html: #if timeout occurs, file gets deleted
        #os.remove(f'{html_game_dir}{new_file}')
        #continue
      f_object.write(str(html))
      print('finished_writing')
      f_object.close()
  print(f'completed {file_number+1} / 208 files')

games_from_march-2004.csv
completed 1 / 208 files
games_from_march-2010.csv
completed 2 / 208 files
games_from_february-1989.csv
visiting https://basketball-reference.com/boxscores/198902010LAC.html
https://basketball-reference.com/boxscores/198902010LAC.html
got html
finished_writing
visiting https://basketball-reference.com/boxscores/198902010PHI.html
https://basketball-reference.com/boxscores/198902010PHI.html
got html
finished_writing
visiting https://basketball-reference.com/boxscores/198902010PHO.html
https://basketball-reference.com/boxscores/198902010PHO.html
got html
finished_writing
visiting https://basketball-reference.com/boxscores/198902010UTA.html
https://basketball-reference.com/boxscores/198902010UTA.html
got html
finished_writing
visiting https://basketball-reference.com/boxscores/198902020CHI.html
Timeout on https://basketball-reference.com/boxscores/198902020CHI.html
https://basketball-reference.com/boxscores/198902020CHI.html
got html
finished_writing
visiting https

CancelledError: 

Functions that will convert str data type into correct, useful data types

In [7]:
def is_home_team(num):
    return int(num)

def get_term(num):
    return terms[int(num)]

def min_to_sec(time):        #converts time to an integer
    if time == 'Did Not Play' or time == 'Did Not Dress' or time is None or time == '\xa0':
        return 0
    
    time_list = time.split(':')
    return int(time_list[0]) * 60 + int(time_list[1])

def int_conv(num):           #converts number to int
    if num == '':
        return None
    
    return int(num)

def float_conv(num):       #converts number to float to nearest 1000th place, rounding down
    if num == '':
        return None
    
    return float(Decimal(str(num)).quantize(Decimal('.001'), rounding=ROUND_DOWN))

def plus_minus_conv(val):
    if val == '':
        return None
    if val == '0':
        return 0
    
    value_symbol = val[0]
    num = int(val[1:])
    if value_symbol == '-':
        return -num
    else:
        return num


converters = [is_home_team, get_term, str, min_to_sec, int_conv, int_conv, float_conv, int_conv, int_conv, float_conv,
              int_conv, int_conv, float_conv, int_conv, int_conv, int_conv, int_conv, int_conv,
              int_conv, int_conv, int_conv, int_conv, plus_minus_conv]


directory = 'Collecting_Data/game_html'
terms = ['Q1', 'Q2', 'H1', 'Q3', 'Q4', 'H2']
headers = ['Home Team', 'Term', 'Name', 'Time Played (sec)', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
           'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']

Writes stats for every player in every game, separated by quarter, to a .csv format with correct data types

In [9]:
for file_number, filename in enumerate(os.listdir(directory)):
    new_file = f'cleaned_data_{filename[:-5]}.csv'

    if os.path.isfile(f'{curr_dir}/Cleaned_data/{new_file}'):
        os.remove(f'{curr_dir}/Cleaned_data/{new_file}')
    print(filename)

    soup = BeautifulSoup(open(f'{curr_dir}/{directory}/{filename}'), 'html.parser')
    #creates a list of stats per quarter for each team
    tables = soup.find_all('table', class_='sortable stats_table')

    found_regex = [] # contains the id of q1 tables to get the amount of players
    regex = r'box-[A-Z]{3}-q1-basic' #expression used to search for specific tables

    for table in tables: #grabs q1 table for each team to identify number of players
        if re.search(regex, table['id']) and table['id'] not in found_regex:
            found_regex.append(table['id'])



    #will contain all data for every player
    player_data = []
    temp = soup.find_all('table', id= regex)

    print('begin looping')
    #teamo_num == 0 is away team, team_num == 1 is home team
    for team_num in range(2):
        #grab number of players currently on the team
        #subtract 4 because there are two rows of headers for each table, 2 rows * 2 teams = 4 extra rows
        amount_of_players = len(soup.find('table', attrs={'id': found_regex[team_num]}).find_all('tr')) - 4
        
        for player_number in range(amount_of_players): #loops through every player on the team
            for index in range(6): #gets all stats for every term for that player
                quarter_stats = tables[index + (team_num*6)].find('tbody').find_all('tr', attrs={'class': None})[player_number]
                #print(quarter_stats)
                data_row = f'{is_home_team(team_num)}, {index},' + ','.join(e.text for e in quarter_stats.children)
                player_data.append(data_row)

    
    with open(f'{curr_dir}/Cleaned_data/{new_file}', 'w') as f_object: #Write headers to every file needed
        writer_object = writer(f_object)
        writer_object.writerow(headers)
        f_object.close()

    
    quarter_index = 0

    for i, n in enumerate(player_data):  #separates stats for each player from every term
        '''
        if (quarter_index) % ((len(tables) // 2)) == 0:
            print('end of player stats')
        quarter_index += 1
        '''
        results = []
        row = n.split(',')
        
        for i, func in enumerate(converters): #converts stats to correct type of data
            if len(row) > 4:
                converted_val = func(row[i])
                results.append(converted_val)
            else:
                results.append(is_home_team(row[0]))
                results.append(get_term(row[1]))
                results.append(row[2])
                results.append(None)
                break

        print(results)
        with open(f'{curr_dir}/Cleaned_data/{new_file}', 'a') as f_object: #writes player data to new file
            writer_object = writer(f_object)
            writer_object.writerow(results)
            f_object.close()

200611280SAC.html
begin looping
[0, 'Q1', 'Elton Brand', 720, 3, 6, 0.5, 0, 0, None, 2, 2, 1.0, 5, 2, 7, 0, 0, 0, 0, 0, 8, 0]
[0, 'Q2', 'Elton Brand', 373, 1, 3, 0.333, 0, 0, None, 1, 1, 1.0, 1, 2, 3, 0, 0, 0, 0, 0, 3, -3]
[0, 'H1', 'Elton Brand', 1093, 4, 9, 0.444, 0, 0, None, 3, 3, 1.0, 6, 4, 10, 0, 0, 0, 0, 0, 11, -3]
[0, 'Q3', 'Elton Brand', 720, 0, 5, 0.0, 0, 0, None, 2, 2, 1.0, 1, 4, 5, 1, 0, 0, 1, 2, 2, -1]
[0, 'Q4', 'Elton Brand', 583, 0, 1, 0.0, 0, 0, None, 0, 0, None, 1, 2, 3, 0, 0, 1, 1, 2, 0, -11]
[0, 'H2', 'Elton Brand', 1303, 0, 6, 0.0, 0, 0, None, 2, 2, 1.0, 2, 6, 8, 1, 0, 1, 2, 4, 2, -12]
[0, 'Q1', 'Shaun Livingston', 720, 5, 6, 0.833, 0, 0, None, 0, 0, None, 0, 1, 1, 2, 0, 0, 1, 1, 10, 0]
[0, 'Q2', 'Shaun Livingston', 372, 1, 2, 0.5, 0, 0, None, 0, 0, None, 1, 0, 1, 0, 0, 0, 0, 1, 2, -5]
[0, 'H1', 'Shaun Livingston', 1092, 6, 8, 0.75, 0, 0, None, 0, 0, None, 1, 1, 2, 2, 0, 0, 1, 2, 12, -5]
[0, 'Q3', 'Shaun Livingston', 720, 4, 5, 0.8, 0, 0, None, 0, 1, 0.0, 0, 2, 2, 1,

IndexError: list index out of range